In [1]:
from datetime import datetime as date
from datetime import timedelta
import pandas as pd
from sportsipy.ncaab.boxscore import Boxscore
from sportsipy.ncaab.boxscore import Boxscores 

In [2]:
games_22 = pd.read_csv('./Kaggle_Datasets/sportsref22data.csv')

In [3]:
def get_current_games(start_date, end_date):
    
    games = Boxscores(start_date, end_date)
    schedule = games.games

    game_days = []
    for day in schedule.values():
        for game in day:
            game_days.append(game['boxscore'])
            
    season_games = []

    for i in game_days:
        temp = Boxscore(i).dataframe
        season_games.append(temp)

    df = pd.concat(season_games, axis = 0).reset_index()

    return df

In [9]:
last_fetch = date(2021,12,9)
yesterday = date.today() - timedelta(days = 1)

df = get_current_games(start_date=last_fetch, end_date=yesterday)

In [10]:
winner_cols = ['index', 'date', 'winning_name', 'losing_name', 'winner',
    'location', 'home_field_goals','home_field_goal_attempts','home_three_point_field_goals', 
    'home_three_point_field_goal_attempts', 'home_free_throws', 'home_free_throw_attempts', 
    'home_offensive_rebounds', 'home_defensive_rebounds', 'home_assists', 
    'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls','away_field_goals',
    'away_field_goal_attempts', 'away_three_point_field_goals', 'away_three_point_field_goal_attempts', 
    'away_free_throws', 'away_free_throw_attempts', 'away_offensive_rebounds', 'away_defensive_rebounds', 
    'away_assists', 'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls']

away = ['index', 'date', 'winning_name', 'losing_name', 'away_points', 'home_points', 'winner', 
    'location', 'away_field_goals','away_field_goal_attempts', 'away_three_point_field_goals', 
    'away_three_point_field_goal_attempts', 'away_free_throws', 'away_free_throw_attempts',
    'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists','away_field_goal_attempts', 
    'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls','home_field_goals',
    'home_field_goal_attempts', 'home_three_point_field_goals', 'home_three_point_field_goal_attempts', 
    'home_free_throws', 'home_free_throw_attempts', 'home_offensive_rebounds', 'home_defensive_rebounds', 
    'home_assists', 'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls']

In [11]:
def prepare_for_team_function(df, year=2022):
    
    df.drop(['Unnamed: 0', 'level_0'], axis = 1, inplace = True)
    
    work_df = df[['index', 'date', 'winning_name', 'losing_name', 'away_points', 'home_points', 'winner', 
    'location', 'away_field_goals','away_field_goal_attempts', 'away_three_point_field_goals', 
    'away_three_point_field_goal_attempts', 'away_free_throws', 'away_free_throw_attempts',
    'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists', 
    'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls','home_field_goals',
    'home_field_goal_attempts', 'home_three_point_field_goals', 'home_three_point_field_goal_attempts', 
    'home_free_throws', 'home_free_throw_attempts', 'home_offensive_rebounds', 'home_defensive_rebounds', 
    'home_assists', 'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls']]
    
    home_df = work_df.filter(like = 'home')
    away_df = work_df.filter(like = 'away')
    info_df = work_df[['index','date','winning_name', 'losing_name', 'winner', 'location']]
    
    winner_cols = ['index', 'date', 'winning_name', 'losing_name', 'winner', 'location',
       'home_points', 'home_field_goals', 'home_field_goal_attempts',
       'home_three_point_field_goals', 'home_three_point_field_goal_attempts',
       'home_free_throws', 'home_free_throw_attempts',
       'home_offensive_rebounds', 'home_defensive_rebounds', 'home_assists',
       'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls',
       'away_points', 'away_field_goals', 'away_field_goal_attempts',
       'away_three_point_field_goals', 'away_three_point_field_goal_attempts',
       'away_free_throws', 'away_free_throw_attempts',
       'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists',
       'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls']

    away = ['index', 'date', 'winning_name', 'losing_name', 'winner', 'location',
       'away_points', 'away_field_goals', 'away_field_goal_attempts',
       'away_three_point_field_goals', 'away_three_point_field_goal_attempts',
       'away_free_throws', 'away_free_throw_attempts',
       'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists',
       'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls',
       'home_points', 'home_field_goals', 'home_field_goal_attempts',
       'home_three_point_field_goals', 'home_three_point_field_goal_attempts',
       'home_free_throws', 'home_free_throw_attempts',
       'home_offensive_rebounds', 'home_defensive_rebounds', 'home_assists',
       'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls']

    new_cols = dict(zip(winner_cols, away))
    
    games = []

    for i in range(len(work_df)):
        if work_df.winner[i] == 'Home':
            y = pd.concat([info_df.loc[[i]], home_df.loc[[i]], away_df.loc[[i]]], axis = 1)
            games.append(y)
        if work_df.winner[i] == 'Away':
            x = pd.concat([info_df.loc[[i]], away_df.loc[[i]], home_df.loc[[i]]], axis = 1)
            temp2 = x.rename(columns = new_cols)
            games.append(temp2)
    regularseason = pd.concat(games, axis = 0, ignore_index = True)
    
    regularseason['Season'] = year
    
    daynum = []

    for i in range(len(df)):
        daynum.append((datetime.strptime(df['date'][i], '%B %d, %Y') - datetime(2021,11,1)).days)
    
    regularseason['DayNum'] = pd.Series(daynum)
    regularseason['NumOT'] = 0
    
    values = ['WTeamID', 'LTeamID', 'WLoc', 'WScore', 'WFGM', 'WFGA', 'WFGM3',
       'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk',
       'WPF', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
    keys = ['winning_name', 'losing_name', 'winner', 'home_points', 'home_field_goals', 'home_field_goal_attempts', 'home_three_point_field_goals', 
         'home_three_point_field_goal_attempts', 'home_free_throws', 'home_free_throw_attempts', 'home_offensive_rebounds', 'home_defensive_rebounds', 
         'home_assists', 'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls', 'away_points', 'away_field_goals', 'away_field_goal_attempts', 
          'away_three_point_field_goals', 'away_three_point_field_goal_attempts', 'away_free_throws', 'away_free_throw_attempts', 'away_offensive_rebounds',
          'away_defensive_rebounds', 'away_assists','away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls']
    
    bbref_change = dict(zip(keys, values))
    regularseason_ready = regularseason.rename(columns = bbref_change)
    regularseason_ready = regularseason.drop(['Unnamed: 0', 'index', 'location', 'date'], axis = 1)
    
    regularseason_ready = regularseason_ready[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore',  'WLoc', 'NumOT',
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 
    'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA','LOR', 'LDR', 'LAst', 
    'LTO', 'LStl', 'LBlk', 'LPF']]

In [13]:
#df = pd.read_csv('./Kaggle_Datasets/sportsref22data.csv')

work_df = df[['index', 'date', 'winning_name', 'losing_name', 'away_points', 'home_points', 'winner', 
    'location', 'away_field_goals','away_field_goal_attempts', 'away_three_point_field_goals', 
    'away_three_point_field_goal_attempts', 'away_free_throws', 'away_free_throw_attempts',
    'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists', 
    'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls','home_field_goals',
    'home_field_goal_attempts', 'home_three_point_field_goals', 'home_three_point_field_goal_attempts', 
    'home_free_throws', 'home_free_throw_attempts', 'home_offensive_rebounds', 'home_defensive_rebounds', 
    'home_assists', 'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls']]

In [14]:
home_df = work_df.filter(like = 'home')
away_df = work_df.filter(like = 'away')
info_df = work_df[['index','date','winning_name', 'losing_name', 'winner', 'location']]

In [15]:
winner_cols = ['index', 'date', 'winning_name', 'losing_name', 'winner', 'location',
       'home_points', 'home_field_goals', 'home_field_goal_attempts',
       'home_three_point_field_goals', 'home_three_point_field_goal_attempts',
       'home_free_throws', 'home_free_throw_attempts',
       'home_offensive_rebounds', 'home_defensive_rebounds', 'home_assists',
       'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls',
       'away_points', 'away_field_goals', 'away_field_goal_attempts',
       'away_three_point_field_goals', 'away_three_point_field_goal_attempts',
       'away_free_throws', 'away_free_throw_attempts',
       'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists',
       'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls']

away = ['index', 'date', 'winning_name', 'losing_name', 'winner', 'location',
       'away_points', 'away_field_goals', 'away_field_goal_attempts',
       'away_three_point_field_goals', 'away_three_point_field_goal_attempts',
       'away_free_throws', 'away_free_throw_attempts',
       'away_offensive_rebounds', 'away_defensive_rebounds', 'away_assists',
       'away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls',
       'home_points', 'home_field_goals', 'home_field_goal_attempts',
       'home_three_point_field_goals', 'home_three_point_field_goal_attempts',
       'home_free_throws', 'home_free_throw_attempts',
       'home_offensive_rebounds', 'home_defensive_rebounds', 'home_assists',
       'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls']

new_cols = dict(zip(winner_cols, away))

In [16]:
games = []

for i in range(len(work_df)):
    if work_df.winner[i] == 'Home':
        y = pd.concat([info_df.loc[[i]], home_df.loc[[i]], away_df.loc[[i]]], axis = 1)
        games.append(y)
    if work_df.winner[i] == 'Away':
        x = pd.concat([info_df.loc[[i]], away_df.loc[[i]], home_df.loc[[i]]], axis = 1)
        temp2 = x.rename(columns = new_cols)
        games.append(temp2)

In [17]:
regularseason22 = pd.concat(games, axis = 0, ignore_index = True)

In [19]:
#regularseason21 = pd.read_csv('data/regularseason21.csv')

In [ ]:
#regularseason = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
#regularseason.columns

In [20]:
regularseason22

,index,date,winning_name,losing_name,winner,location,home_points,home_field_goals,home_field_goal_attempts,home_three_point_field_goals,...,away_three_point_field_goal_attempts,away_free_throws,away_free_throw_attempts,away_offensive_rebounds,away_defensive_rebounds,away_assists,away_turnovers,away_steals,away_blocks,away_personal_fouls
0,2021-12-09-19-rutgers,"December 9, 2021",Rutgers,Purdue,Home,"Rutgers Athletic Center, New Brunswick, New Je...",70,25,48,7,...,26,11,16,13,17,10,11,8,2,17
1,2021-12-09-21-gonzaga,"December 9, 2021",Gonzaga,Merrimack,Home,"McCarthey Athletic Center, Spokane, Washington",80,30,59,5,...,25,9,11,6,12,8,8,12,0,16
2,2021-12-09-18-seton-hall,"December 9, 2021",Seton Hall,Texas,Home,"Prudential Center, Newark, New Jersey",64,23,57,5,...,13,9,9,10,23,8,10,3,2,18
3,2021-12-09-21-iowa-state,"December 9, 2021",Iowa State,Iowa,Home,"James H. Hilton Coliseum , Ames, Iowa",73,27,62,8,...,27,14,15,13,15,10,12,7,6,18
4,2021-12-09-19-bellarmine,"December 9, 2021",Bellarmine,Asbury\n\t\t\t,Home,"KFC Yum! Center, Louisville, Kentucky",87,34,67,9,...,27,6,10,6,19,10,10,5,3,11
5,2021-12-09-18-north-carolina-state,"December 9, 2021",NC State,Bethune-Cookman,Home,"Reynolds Coliseum, Raleigh, North Carolina",65,24,61,7,...,13,4,6,8,24,6,13,5,3,16
6,2021-12-09-19-north-carolina-central,"December 9, 2021",North Carolina Central,Carver College\n\t\t\t,Home,"McLendon-McDougald Gym, Durham, North Carolina",102,37,64,9,...,20,9,12,1,16,11,25,2,2,22
7,2021-12-09-20-pittsburgh,"December 9, 2021",Pitt,Colgate,Home,"Petersen Events Center, Pittsburgh, Pennsylvania",71,24,55,4,...,23,18,23,8,15,13,10,5,1,21
8,2021-12-09-21-new-mexico,"December 9, 2021",New Mexico,Denver,Home,"The Pit, Albuquerque, New Mexico",87,27,62,7,...,19,10,17,6,22,15,8,4,2,18
9,2021-12-09-19-kent-state,"December 9, 2021",Kent State,Detroit,Home,"Memorial Athletic & Convocation Center, Kent, ...",69,23,52,8,...,32,10,12,8,17,12,9,5,2,15


In [21]:
df = regularseason22

In [22]:
df['Season'] = 2022

In [24]:
DayNum = []

for i in range(len(df)):
    DayNum.append((date.strptime(df['date'][i], '%B %d, %Y') - date(2021,11,1)).days)
    
df['DayNum'] = pd.Series(DayNum)

In [25]:
df['NumOT'] = 0

In [26]:
values = ['WTeamID', 'LTeamID', 'WLoc', 'WScore', 'WFGM', 'WFGA', 'WFGM3',
       'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk',
       'WPF', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

In [27]:
keys = ['winning_name', 'losing_name', 'winner', 'home_points', 'home_field_goals', 'home_field_goal_attempts', 'home_three_point_field_goals', 
         'home_three_point_field_goal_attempts', 'home_free_throws', 'home_free_throw_attempts', 'home_offensive_rebounds', 'home_defensive_rebounds', 
         'home_assists', 'home_turnovers', 'home_steals', 'home_blocks', 'home_personal_fouls', 'away_points', 'away_field_goals', 'away_field_goal_attempts', 
          'away_three_point_field_goals', 'away_three_point_field_goal_attempts', 'away_free_throws', 'away_free_throw_attempts', 'away_offensive_rebounds',
          'away_defensive_rebounds', 'away_assists','away_turnovers', 'away_steals', 'away_blocks', 'away_personal_fouls']

In [28]:
bbref_change = dict(zip(keys,values))
bbref_change

{'winning_name': 'WTeamID',
 'losing_name': 'LTeamID',
 'winner': 'WLoc',
 'home_points': 'WScore',
 'home_field_goals': 'WFGM',
 'home_field_goal_attempts': 'WFGA',
 'home_three_point_field_goals': 'WFGM3',
 'home_three_point_field_goal_attempts': 'WFGA3',
 'home_free_throws': 'WFTM',
 'home_free_throw_attempts': 'WFTA',
 'home_offensive_rebounds': 'WOR',
 'home_defensive_rebounds': 'WDR',
 'home_assists': 'WAst',
 'home_turnovers': 'WTO',
 'home_steals': 'WStl',
 'home_blocks': 'WBlk',
 'home_personal_fouls': 'WPF',
 'away_points': 'LScore',
 'away_field_goals': 'LFGM',
 'away_field_goal_attempts': 'LFGA',
 'away_three_point_field_goals': 'LFGM3',
 'away_three_point_field_goal_attempts': 'LFGA3',
 'away_free_throws': 'LFTM',
 'away_free_throw_attempts': 'LFTA',
 'away_offensive_rebounds': 'LOR',
 'away_defensive_rebounds': 'LDR',
 'away_assists': 'LAst',
 'away_turnovers': 'LTO',
 'away_steals': 'LStl',
 'away_blocks': 'LBlk',
 'away_personal_fouls': 'LPF'}

In [29]:
df = df.rename(columns = bbref_change)
df = df.drop(['index', 'location', 'date'], axis = 1)
df.columns

Index(['WTeamID', 'LTeamID', 'WLoc', 'WScore', 'WFGM', 'WFGA', 'WFGM3',
       'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk',
       'WPF', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'Season', 'DayNum',
       'NumOT'],
      dtype='object')

In [30]:
df = df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore',  'WLoc', 'NumOT',
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 
    'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA','LOR', 'LDR', 'LAst', 
    'LTO', 'LStl', 'LBlk', 'LPF']]
       

In [31]:
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2022,38,Rutgers,70,Purdue,68,Home,0,25,48,...,26,11,16,13,17,10,11,8,2,17
1,2022,38,Gonzaga,80,Merrimack,55,Home,0,30,59,...,25,9,11,6,12,8,8,12,0,16
2,2022,38,Seton Hall,64,Texas,60,Home,0,23,57,...,13,9,9,10,23,8,10,3,2,18
3,2022,38,Iowa State,73,Iowa,53,Home,0,27,62,...,27,14,15,13,15,10,12,7,6,18
4,2022,38,Bellarmine,87,Asbury\n\t\t\t,55,Home,0,34,67,...,27,6,10,6,19,10,10,5,3,11
5,2022,38,NC State,65,Bethune-Cookman,48,Home,0,24,61,...,13,4,6,8,24,6,13,5,3,16
6,2022,38,North Carolina Central,102,Carver College\n\t\t\t,50,Home,0,37,64,...,20,9,12,1,16,11,25,2,2,22
7,2022,38,Pitt,71,Colgate,68,Home,0,24,55,...,23,18,23,8,15,13,10,5,1,21
8,2022,38,New Mexico,87,Denver,67,Home,0,27,62,...,19,10,17,6,22,15,8,4,2,18
9,2022,38,Kent State,69,Detroit,52,Home,0,23,52,...,32,10,12,8,17,12,9,5,2,15


In [32]:
df.shape

(31, 34)

In [33]:
kenpom = pd.read_csv('./Data/KenPom20212109.csv')
team_ids = pd.read_csv('./Data/TeamNames.csv')
team_ids_sportsipy = dict(zip(team_ids.Sportsipy,team_ids.TeamID))
team_ids_kenpom = dict(zip(team_ids.KenPom,team_ids.TeamID))
kenpom = kenpom.rename(columns = {'RankAdjEM' : 'OrdinalRank', 'TeamName': 'TeamID'})
kenpom.TeamID = kenpom['TeamID'].replace(team_ids_dict)

NameError: name 'team_ids_dict' is not defined

In [34]:
df['WTeamID'] = df['WTeamID'].replace(team_ids_sportsipy)
df['LTeamID'] = df['LTeamID'].replace(team_ids_sportsipy)

In [35]:
df.WTeamID = df.WTeamID.astype(str).str.extract('(\d+)').astype(float, errors = 'ignore')

In [36]:
df.LTeamID = df.LTeamID.astype(str).str.extract('(\d+)').astype(float, errors = 'ignore')

In [37]:
df.shape

(31, 34)

In [38]:
df.dropna(inplace = True)
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2022,38,1353.0,70,1345.0,68,Home,0,25,48,...,26,11,16,13,17,10,11,8,2,17
1,2022,38,1211.0,80,1467.0,55,Home,0,30,59,...,25,9,11,6,12,8,8,12,0,16
2,2022,38,1371.0,64,1400.0,60,Home,0,23,57,...,13,9,9,10,23,8,10,3,2,18
3,2022,38,1235.0,73,1234.0,53,Home,0,27,62,...,27,14,15,13,15,10,12,7,6,18
5,2022,38,1301.0,65,1126.0,48,Home,0,24,61,...,13,4,6,8,24,6,13,5,3,16
7,2022,38,1338.0,71,1159.0,68,Home,0,24,55,...,23,18,23,8,15,13,10,5,1,21
8,2022,38,1307.0,87,1176.0,67,Home,0,27,62,...,19,10,17,6,22,15,8,4,2,18
9,2022,38,1245.0,69,1178.0,52,Home,0,23,52,...,32,10,12,8,17,12,9,5,2,15
10,2022,38,1113.0,67,1213.0,62,Home,0,19,61,...,32,8,13,19,30,10,11,2,2,19
11,2022,38,1214.0,54,1456.0,53,Away,0,21,53,...,24,11,21,10,28,10,18,6,3,18


In [48]:
df2 = pd.read_csv('./Kaggle_Datasets/regularseason22_raw.csv')
df2.drop(['Unnamed: 0'], axis = 1, inplace = True)
df2.columns
#df.to_csv('./Kaggle_Datasets/regularseason22_raw.csv')

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [49]:
df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [54]:
df = pd.concat([df2, df], axis = 0)

In [55]:
df.to_csv('./Kaggle_Datasets/regularseason22_raw_2021_12_11.csv')

In [274]:
games = Boxscores(datetime(2021,12,11))
schedule = games.games

game_days = []
for day in schedule.values():
    for game in day:
        game_days.append((game['away_name'], game['home_name']))

In [275]:
game_days

[('Appalachian State', 'Furman'),
 ('Army', 'NJIT'),
 ('Cal State Northridge', 'North Dakota State'),
 ('DePaul', 'Louisville'),
 ('Green Bay', 'Kansas City'),
 ('IUPUI', 'Tennessee State'),
 ('Loyola (IL)', 'Vanderbilt'),
 ('Midway', 'Bellarmine'),
 ('Milwaukee', 'Colorado'),
 ('Moravian', 'Delaware'),
 ('Murray State', 'Memphis'),
 ('Prairie View', 'Boise State'),
 ('South Carolina State', 'High Point'),
 ('Vermont', 'Brown'),
 ('Yellowstone Christian', 'Montana')]

In [276]:
type(game['away_name'])

str

In [285]:
x = team_ids_sportsipy.get(game['away_name'])

In [287]:
team_ids_sportsipy.get(game['away_name'])